In [8]:
import folium
import pandas as pd
from tqdm import tqdm

In [9]:
m = folium.Map(location=[52.215933,19.134422], zoom_start=6)
ic_nodes_coord = pd.read_csv('ic_coord.nodes')

def removeAccents(text): # encoding works in html, but not in Jupyter view
    polishChars = 'ąćęłńóśżźĄĆĘŁŃÓŚŻŹ'
    replace     = 'acelnoszzACELNOSZZ'
    translator=str.maketrans(polishChars, replace)
    return text.translate(translator)

In [10]:
data = ic_nodes_coord
for index, row in data.iterrows():

    folium.Marker(
        [row['latitude'],row['longitude']],
        popup = row['name'] # encoding works in html, but not in Jupyter view
#         popup = removeAccents(row['name']) 
#         Popup encoding issue with Jupyter
#         https://github.com/python-visualization/folium/issues/575
    ).add_to(m)

In [11]:
m.save('map_vis_no_edges.html')
m

In [12]:
ic_edges = pd.read_csv('ic.edges')
ic_edges

,source,target,weight
0,0,338,23
1,0,394,21
2,1,147,2
3,1,354,1
4,2,54,4
...,...,...,...
1338,457,362,26
1339,457,363,1
1340,458,14,6
1341,458,101,3


In [13]:
data = ic_edges

with tqdm(total=len(data.index)) as pbar:
    for index, row in data.iterrows():
        source = ic_nodes_coord.loc[ic_nodes_coord['id'] == row['source']]
        target = ic_nodes_coord.loc[ic_nodes_coord['id'] == row['target']]
        line_coord = [(source['latitude'].to_numpy()[0], source['longitude'].to_numpy()[0]),
                (target['latitude'].to_numpy()[0], target['longitude'].to_numpy()[0])]
        popup = source['name'].to_numpy()[0] + ' -> ' + target['name'].to_numpy()[0]
        ic_color = '#0a2255'
        folium.PolyLine(line_coord, popup=popup, weight=2, opacity=0.6, color=ic_color).add_to(m)
        pbar.update(1)   

100%|██████████| 1343/1343 [00:01<00:00, 1235.66it/s]


In [14]:
m.save('map_vis.html')
m